In [ ]:
import eodslib
from pathlib import Path
import subprocess
from dotenv import load_dotenv
import os

In [ ]:
# USER MUST EDIT THE ENVIRONMENT FILE REFERENCED BELOW, OR CREATE THEIR OWN FILE AND REFERENCE IT
load_dotenv('sample.env')

In [ ]:
# set configuration based on contents of the ENVIRONMENT FILE.
conn = {
    'domain': os.getenv("HOST"),
    'username': os.getenv("API_USER"),
    'access_token': os.getenv("API_TOKEN"),
    }

In [ ]:
# use default path to local "output" directory
output_dir = eodslib.make_output_dir(Path.cwd() / 'output')

In [ ]:
# example of EODS query for generating a cloud free mosaic for Summer 2020
# since no 'geom' parameter is set, this will query every granule reference with the catalogue

eods_params = {
    'output_dir':output_dir,
    'sat_id':2,
    'date_start':'2020-05-01',
    'date_end':'2020-06-30',
    'find_least_cloud':True,
    }

list_of_layers, df = eodslib.query_catalog(conn, **eods_params)

In [ ]:
# this is an example where you want to override the list of layers returned by the EODS query cell above
list_of_layers = [
    'geonode:S2B_20200424_lat55lon215_T30UWF_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref',
    'geonode:S2A_20200419_lat55lon061_T30UXF_ORB037_utm30n_osgb_vmsk_sharp_rad_srefdem_stdsref'
]

In [ ]:
# create an empty list of results
list_of_results = list()

# iterative through the returned EODS query list of layers
for lyr in list_of_layers:

    # for each layer, create a wps config dictionary, inserting the layer name
    config_wpsprocess = {'template_xml':'gsdownload_template.xml',
        'xml_config':{
            'template_layer_name':lyr,
            'template_outputformat':'image/tiff',
            'template_mimetype':'application/zip'
                },
        'dl_bool':True
    }

    # for each layer, call the run wps function which:
        # submits the WPS job
        # polls the wps server until the job is READY
        # downloads the result (and retries up to 3 times if there's a break in the download connection)
    execution_dict = eodslib.run_wps(conn, config_wpsprocess, output_dir=output_dir, verify=True)

    # append the execution dictionary to the results list
    list_of_results.append(execution_dict)

# log the output to a log csv file
eodslib.output_log(list_of_results)

In [ ]:
# grab the s2 output files
rio_merge_cmd = [str(input_file) for input_file in output_dir.glob('**/*osgb_vmsk_sharp_rad_srefdem_stdsref.tiff')]

# build a list to represent the rio merge command
rio_merge_cmd.insert(0,'rio')
rio_merge_cmd.insert(1,'merge')
rio_merge_cmd.append(str(output_dir / 'merge-tmp.tiff'))

# execute the the merge of the downloaded S2s to a single geotiff, note this can be a BIG file ... 
subprocess.run(rio_merge_cmd, capture_output=True)

In [ ]:
# build a list to represent the rio overview command and execute it with subprocess
# this adds overviews to the merged output
rio_overviews_cmd = ['rio', 'overview', '--build', '2,4,8,16', str(output_dir / 'merge-tmp.tiff')]
subprocess.run(rio_overviews_cmd, capture_output=True)

In [ ]:
# build a list to represent the rio conver command and execute it with subprocess,
# this adds inner tiling and compression, effectively optimising the rendering in a gis application
rio_convert_cmd = [
    'rio',
    'convert',
    str(output_dir / 'merge-tmp.tiff'),
    str(output_dir / 'merge.tiff'),
    '--co',
    'compress=LZW',
    '--co',
    'tiled=true',
    '--co',
    'blockxsize=256',
    '--co',
    'blockysize=256',
]
subprocess.run(rio_convert_cmd, capture_output=True)

In [ ]:
# drop intermediate merged file
Path(output_dir / 'merge-tmp.tiff').unlink()

In [ ]:
print('### Script Finished')